# GroupChatSweetner

*Project proposed to Pianalytix Edutech Pvt Ltd.*

**Team 7.5**

    - Deepak Sahu ( deepak@iiitkalyani.ac.in)
    - Devendra Singh (devendrachauhan299@gmail.com)
    - Jeevitha S (jeevithasuresh2000@gmail.com)
    - Kishan Sahu  (sahukishan06@gmail.com)
    - Nadine Ben Harrath (nadinebenharrath@gmail.com)
    - Rakshitha D (rakshupaapu1999@gmail.com)
    - Saurav Chaudhary (sauravchaudhary_bt18@iiitkalyani.ac.in)

Period of Internship: **November (2020)**

**Keywords**

    NLP, Multi-label Classification, Text/Chat-based group messaging moderator, Live Topic filtering


# Project brief

---
This is demonstration of part of the project/concept **GroupChatSweetner**.
- The project is aimed towards classifying a `casual` conversation from the conversations related to topics `politics`, `crime`, `religion`, `bollywood`.
-so that text-based group conversation could be moderated
- as not every social group is mean for conversation on every topic
---
# Notebook Brief
In this notebook we had demonstrated topic classification on **custom reddit scrapped** conversation/titles relavent to the topics we need to classify.

> Although the project implementation would require multi-label classification, for simplicity we had started with multi-class classification.
---

**IMPORTANT**

Please run this notebook in google-colab.

## Loading Data

Data had been scrapped from reddit for labels belonging to `politics`, `crime`, `religion`, `bollywood`.

But for getting data on `casual` conversation we had subset the untagged data from https://convokit.cornell.edu/documentation/reddit-small.html

In [ ]:
# Loading required libraries
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/reddit.csv')
df.tail()

,text,label
4470,An interesting interpretation of Mani Ratnam's...,bollywood
4471,Hindi/Bollywood Laal Dupatta song - can anyone...,bollywood
4472,Yeh bik gayi hain IMDB,bollywood
4473,Ashok Saraf | With 250+ Marathi films to his c...,bollywood
4474,Abbas-Mustan Interview | The 90s Show | Anupam...,bollywood


In [ ]:
# Corpus size per label
df['label'].value_counts()

casual       974
crime        916
religion     888
bollywood    863
politics     834
Name: label, dtype: int64

## Text PreProcessing

- stopwords removal
- non-alphanumeric character removal
- stemmatization

### Defining functions to preprocess text

In [ ]:
# Loading Libraries
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words=set(stopwords.words('english'))
stop_words.remove('not')

def remove_stopwords(text):
    '''
        function to remove stopwords
    '''
    text=[word for word in text.split(' ') if not word in stop_words]
    return " ".join(text)

# # TESTING
# text = 'This is an ugly line composed of words'
# text_processed = remove_stopwords(text)
# print(f'BEFORE: {text}\n AFTER: {text_processed}')

In [ ]:
def remove_special_char(text):
    '''
        function to remove special characters (non-alphanumeric) from text
    '''
    text_proc = []
    for word in text.split(' '):
        word_proc = re.sub('[^A-Za-z0-9]',' ',word)
        text_proc.append(word_proc)
    text_proc = ' '.join(text_proc)
    text_proc = re.sub('\s+',' ', text_proc) # replaces multiple spaces
    return text_proc

# TESTING
# text = 'This is an ugly:( line composed of stop-words'
# text_processed = remove_special_char(text)
# print(f'BEFORE: {text}\n AFTER: {text_processed}')

In [ ]:
def stemmer(text):
    '''
        function to stemmatize text (remove lingual inflection)
    '''
    stemmer = PorterStemmer()
    text = text.lower()
    text = [stemmer.stem(word) for word in text.split(' ')]
    return ' '.join(text)

# # TESTING
# text = 'He went to the market for marketing when it was raining'
# text_processed = stemmer(text)
# print(f'BEFORE: {text}\n AFTER: {text_processed}')

### Applying Text preprocessing

In [ ]:
df['text_processed'] = df['text'].apply(remove_stopwords)
df['text_processed'] = df['text_processed'].apply(remove_special_char)
df['text_processed'] = df['text_processed'].apply(stemmer)
df.tail()

,text,label,text_processed
4470,An interesting interpretation of Mani Ratnam's...,bollywood,an interest interpret mani ratnam s raavan
4471,Hindi/Bollywood Laal Dupatta song - can anyone...,bollywood,hindi bollywood laal dupatta song anyon help find
4472,Yeh bik gayi hain IMDB,bollywood,yeh bik gayi hain imdb
4473,Ashok Saraf | With 250+ Marathi films to his c...,bollywood,ashok saraf with 250 marathi film credit saraf...
4474,Abbas-Mustan Interview | The 90s Show | Anupam...,bollywood,abba mustan interview the 90 show anupama chopra


## Building Classifier using Transfer Learning

The following is an implementation of Bert-uncased pretrained model to do classification over our dataset.

In [ ]:
# Loading libraries
import numpy as np
from tabulate import tabulate
import torch
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset

try:
    from transformers import BertTokenizer
except Exception as e:
    !pip install transformers
finally:
    from transformers import BertTokenizer

from transformers import BertForSequenceClassification
from sklearn.metrics import accuracy_score, confusion_matrix

     |████████████████████████████████| 1.4MB 20.2MB/s 
     |████████████████████████████████| 2.9MB 50.3MB/s 
     |████████████████████████████████| 890kB 55.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=92e8da21f26a70e2c11c85de024be67e3483e39365e3871d28e1fb48d4ceb92f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
df.head()

,text,label,text_processed
0,"Talk about your day. Anything goes, but subred...",casual,talk day anyth goe subreddit rule still appli ...
1,I went to visit a few days ago and Ioved it. I...,casual,i went visit day ago iov it i can t find neg s...
2,"One time, my family and I had just returned fr...",casual,one time famili i return japan need big cab lo...
3,"Talk about your day. Anything goes, but subred...",casual,talk day anyth goe subreddit rule still appli ...
4,"Talk about your day. Anything goes, but subred...",casual,talk day anyth goe subreddit rule still appli ...


### Label Encoding

In [ ]:
# Dictionary of Label-Encoding and decoding
label_dict = {}
label_dict_rev = {}
#
possible_labels = df.label.unique()
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
    label_dict_rev[index] = possible_label
label_dict

{'bollywood': 4, 'casual': 0, 'crime': 2, 'politics': 1, 'religion': 3}

In [ ]:
# Label Encoding
df['label_encoded'] = df.label.replace(label_dict)
df.head()

,text,label,text_processed,label_encoded
0,"Talk about your day. Anything goes, but subred...",casual,talk day anyth goe subreddit rule still appli ...,0
1,I went to visit a few days ago and Ioved it. I...,casual,i went visit day ago iov it i can t find neg s...,0
2,"One time, my family and I had just returned fr...",casual,one time famili i return japan need big cab lo...,0
3,"Talk about your day. Anything goes, but subred...",casual,talk day anyth goe subreddit rule still appli ...,0
4,"Talk about your day. Anything goes, but subred...",casual,talk day anyth goe subreddit rule still appli ...,0


### Train-Val-Test-Split

In [ ]:
# Creating train-test split 70-15-15
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label_encoded.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
X_val, X_test, y_val, y_test = train_test_split(df.loc[X_val].index.values, 
                                                df.loc[X_val].label_encoded.values,
                                                  test_size=0.5, 
                                                  random_state=42, 
                                                  stratify=df.loc[X_val].label.values)
# Data split info
data = [
        ('set', 'shape[0]'),
        ('train', X_train.shape[0]),
        ('val', X_val.shape[0]),
        ('test', X_test.shape[0]),
]
print(tabulate(data, headers='firstrow'))

set      shape[0]
-----  ----------
train        3132
val           671
test          672


In [ ]:
# Classwise-split info
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.loc[X_test, 'data_type'] = 'test'

df.groupby(['label', 'label_encoded', 'data_type']).count()

text  text_processed
label     label_encoded data_type                      
bollywood 4             test        129             129
                        train       604             604
                        val         130             130
casual    0             test        146             146
                        train       682             682
                        val         146             146
crime     2             test        138             138
                        train       641             641
                        val         137             137
politics  1             test        125             125
                        train       584             584
                        val         125             125
religion  3             test        134             134
                        train       621             621
                        val         133             133

### Tokenization

In [ ]:
# Tokenization 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_processed.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=30, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_processed.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=30, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].text_processed.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=30, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label_encoded.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label_encoded.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label_encoded.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

### Model Building

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Sample for batches
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32 #@param {type:"slider", min:16, max:128, step:2}
#
# Batch feeder to model
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
# Selecting gradient descent
from transformers import AdamW, get_linear_schedule_with_warmup

learning_rate = 1e-5 #@param {type: "number" }

optimizer = AdamW(model.parameters(),
                  lr=learning_rate,
                  eps=1e-8)

In [ ]:
epochs = 5  #@param {type: "slider", min:2, max:10, step:2}

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
# Defining Model performance metrics
#
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    '''
        Function to calculate weighted f1 score
    '''
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):
    '''
        function to evaluate model performance
    '''

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

### Model Training

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    # Save model when it had reached total epochs
    if epoch == epochs:
        torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.2031390046586796
Validation loss: 0.7406668762365977
F1 Score (Weighted): 0.8131930457150398



Epoch 2
Training loss: 0.5931311210199278
Validation loss: 0.46312926212946576
F1 Score (Weighted): 0.8567922144246678



Epoch 3
Training loss: 0.3803476204677504
Validation loss: 0.3853952678896132
F1 Score (Weighted): 0.886975020840054



Epoch 4
Training loss: 0.2874478661284155
Validation loss: 0.35384865424462725
F1 Score (Weighted): 0.8913529936143504



Epoch 5
Training loss: 0.2312842924527976
Validation loss: 0.34498756130536395
F1 Score (Weighted): 0.8943853750391936



### Benchmarking Trained Model performance

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.load_state_dict(torch.load(f'finetuned_BERT_epoch_{epochs}.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Train set benchmark
_, predictions, true_vals = evaluate(dataloader_train)
preds = np.argmax(predictions, axis=1)
accuracy_score(true_vals, preds)

0.9639208173690932

In [ ]:
confusion_matrix(true_vals, preds)

array([[647,   3,   5,  18,   9],
       [  0, 573,   8,   3,   0],
       [  2,   6, 623,   8,   2],
       [ 11,   2,   0, 605,   3],
       [ 14,   2,   8,   9, 571]])

In [ ]:
# Val
_, predictions, true_vals = evaluate(dataloader_validation)
preds = np.argmax(predictions, axis=1)
accuracy_score(true_vals, preds)

0.8941877794336811

In [ ]:
# Test
_, predictions, true_vals = evaluate(dataloader_test)
preds = np.argmax(predictions, axis=1)
accuracy_score(true_vals, preds)

0.8943452380952381

## Making New Predicition

In [ ]:
def predict(dataloader_):
    '''
        function to classify new conversations using trained model
    '''

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         None,
                 }

        with torch.no_grad():        
            outputs = model(**inputs)

        # loss = outputs[0]
        logits = outputs[0]
        # loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    predictions = np.concatenate(predictions, axis=0)       
    return predictions

In [ ]:
#@title Test with Custom Input { run: "auto" }
text = "Which movie has highest IMDB rating?" #@param {type:"string"}
#
# Applying pre-processing
text = remove_stopwords(text)
text = remove_special_char(text)
text = stemmer(text)
text = [text, ]
# Encoding
encoded_data_test = tokenizer.batch_encode_plus(
    text,
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',
    max_length=30, 
    return_tensors='pt'
)
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
dataset_test = TensorDataset(input_ids_test, attention_masks_test)
#
# Dataloader
dataloader_test = DataLoader(dataset_test, 
                             sampler=None,
                             shuffle=False,
                             batch_size=1)
# Predictions
predictions = predict(dataloader_test)
pred_label = np.argmax(predictions, axis=1)
print(f'Predicted Class: {label_dict_rev[pred_label[0]]}')

Predicted Class: bollywood


## Future Work
1.This model predicts for english sentences for now.In future model can be extended with multilingual sentences.

2.More topics/labels can be added in future.

3.Conversation can belong to multiple topics.In future model can be transform to multilabel classification.

